# Movie Revenue Predictions

Drie concept-onderzoeksvragen:
- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
- 2
- 3

## Het Data Science proces
Voor de eerste verkenning is ons gevraagd om de eerste vier stappen uit te voeren:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
6. Communication


## 1. Data Collection

## 2. Data Processing

## 3. Data Cleaning

## 4. Data Exploration & Analysis

In [ ]:
import pandas as pd
df = pd.read_csv('movie.csv')
df.describe()

In [ ]:
df.sample(5)

In [ ]:
df.columns